In [1]:
import urllib.request as req
from urllib.parse import unquote
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import json
import xmltodict

In [2]:
url = 'http://api.visitkorea.or.kr/openapi/service/rest/DataLabService/metcoRegnVisitrDDList'
params = dict()
API_KEY = '0nvWRjDWerwQGQ7dGwJhCm%2FxEkkbYBiWQXfnf%2BJkkO3dkoTpaGvyy44heAG1hY%2BjV7ZQxbK0LlDdoeer2yORHw%3D%3D'
KEY = unquote(API_KEY)
params['serviceKey'] = KEY
params['MobileOS'] = 'ETC'
params['MobileApp'] = 'TEST'
params['numOfRows'] = 1000
params['pageNo'] = 1
params['startYmd'] = 20210101
params['endYmd'] = 20210701
params['touDivCd'] = 1

response = requests.get(url,params=params)
# http://api.visitkorea.or.kr/openapi/service/rest/DataLabService/metcoRegnVisitrDDList?serviceKey=서비스인증키
#     &MobileOS=ETC&MobileApp=AppTest
#     &startYmd=20210513&endYmd=2120513

In [3]:
html = response.text
dicts = xmltodict.parse(html)
json_obj = json.dumps(dicts)
dict_data =json.loads(json_obj)
dict_data1 = dict_data['response']['body']['items']['item']
print(dict_data1[0])

{'areaCode': '27', 'areaNm': '대구광역시', 'baseYmd': '20210428', 'daywkDivCd': '3', 'daywkDivNm': '수요일', 'touDivCd': '3', 'touDivNm': '외국인(c)', 'touNum': '260.1273705'}


In [4]:
cols = ['시도코드','시도명','기준연월일','요일코드','요일명','관광객구분코드','관광객구분명','관광객수']
t_df = pd.DataFrame(columns=cols)
idx = 1
for i in range(0,len(dict_data1)):
    areaCode = dict_data1[i]['areaCode']
    areaNm = dict_data1[i]['areaNm']
    baseYmd = dict_data1[i]['baseYmd']
    daywkDivCd = dict_data1[i]['daywkDivCd']
    daywkDivNm = dict_data1[i]['daywkDivNm']
    touDivCd = dict_data1[i]['touDivCd']
    touDivNm = dict_data1[i]['touDivNm']
    touNum = dict_data1[i]['touNum']
    t_df.loc[idx] = [areaCode,areaNm,baseYmd,daywkDivCd,daywkDivNm,touDivCd,touDivNm,touNum]
    idx += 1

In [5]:
t_df = t_df.sort_values('시도코드')
t_df.set_index('시도명')

,시도코드,기준연월일,요일코드,요일명,관광객구분코드,관광객구분명,관광객수
시도명,,,,,,,
서울특별시,11,20210429,4,목요일,1,현지인(a),4492117
서울특별시,11,20210502,7,일요일,3,외국인(c),3323.692794
서울특별시,11,20210502,7,일요일,2,외지인(b),1211213.5
서울특별시,11,20210502,7,일요일,1,현지인(a),4079169.5
서울특별시,11,20210506,4,목요일,1,현지인(a),4724202
...,...,...,...,...,...,...,...
제주특별자치도,50,20210426,1,월요일,1,현지인(a),525175
제주특별자치도,50,20210426,1,월요일,2,외지인(b),184117.5
제주특별자치도,50,20210426,1,월요일,3,외국인(c),1246.437062


In [6]:
from pandas.io.json import json_normalize

In [7]:
url ='http://api.visitkorea.or.kr/openapi/service/rest/KorService/searchFestival?serviceKey=0nvWRjDWerwQGQ7dGwJhCm%2FxEkkbYBiWQXfnf%2BJkkO3dkoTpaGvyy44heAG1hY%2BjV7ZQxbK0LlDdoeer2yORHw%3D%3D&MobileOS=ETC&MobileApp=AppTest&arrange=A&numOfRows=3000&pageNo=1&listYN=Y&_type=json'
response = req.urlopen(url)
json_str = response.read().decode('utf-8')
json_obj = json.loads(json_str)
a = [json_obj['response']['body']['items']['item']]
raw_data = json_normalize(json_obj['response']['body']['items']['item'])

<ipython-input-7-9dadcf69429c>:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  raw_data = json_normalize(json_obj['response']['body']['items']['item'])


In [8]:
data = raw_data.copy()

In [9]:
data.head()
data.columns

Index(['addr1', 'areacode', 'cat1', 'cat2', 'cat3', 'contentid',
       'contenttypeid', 'createdtime', 'eventenddate', 'eventstartdate',
       'firstimage', 'firstimage2', 'mapx', 'mapy', 'mlevel', 'modifiedtime',
       'readcount', 'sigungucode', 'tel', 'title', 'addr2'],
      dtype='object')

In [10]:
#column 정리
data.drop(['areacode','cat1','cat2','cat3','contentid','contenttypeid','createdtime','firstimage','firstimage2','mlevel'],axis=1,inplace=True)

In [11]:
data.drop('addr2',axis=1,inplace=True)

In [12]:
#column 정렬
data.columns

Index(['addr1', 'eventenddate', 'eventstartdate', 'mapx', 'mapy',
       'modifiedtime', 'readcount', 'sigungucode', 'tel', 'title'],
      dtype='object')

In [13]:
new_cols =['title', 'eventstartdate', 'eventenddate','sigungucode', 'mapx', 'mapy', 'tel', 'addr1','readcount','modifiedtime']
data = data[new_cols]
data.set_index('title')

,eventstartdate,eventenddate,sigungucode,mapx,mapy,tel,addr1,readcount,modifiedtime
title,,,,,,,,,
가무악극으로 만나는 토요 상설공연,20210306,20211030,6.0,126.899702,36.304843,041-832-5765,충청남도 부여군 규암면 백제문로 388,56458.0,20210226112411
가산수피아 '봄의 정원',20210320,20210630,22.0,128.485069,36.086155,054-971-9861,경상북도 칠곡군 가산면 학하들안2길 105,0.0,20210409093816
가족끼리 빙어체험 2021,20210116,20211231,19.0,127.657414,37.5145504080,031-775-5205,경기도 양평군 단월면 곱다니길 55-2 수미마을 방문객센터,0.0,20210119152643
가족끼리딸기 캠피닉,20210301,20210606,19.0,127.6573663730,37.514495,031-775-5205,경기도 양평군 단월면 곱다니길 55-2,94324.0,20210308133050
가족오페라 <마술피리> 2021,20210225,20210228,5.0,128.5922570230,35.883507,053-666-6170,대구광역시 북구 호암로 15,0.0,20210201173042
...,...,...,...,...,...,...,...,...,...
[문화관광축제] 추억의 충장축제 2021,20211013,20211017,3.0,126.923129,35.145497,062-608-2421~4,광주광역시 동구 서남로 1,107608.0,20210218164140
[문화관광축제] 춘천마임축제,20210523,20211130,13.0,127.717856,37.876601,033-242-0585,강원도 춘천시 전역,93802.0,20210513085200
[문화관광축제] 통영한산대첩축제,20210812,20210821,17.0,128.424172,34.84383,055-644-5222,경상남도 통영시 통영해안로 328,100521.0,20210604173154


In [14]:
data = data.sort_values('readcount',ascending=False)
data

,title,eventstartdate,eventenddate,sigungucode,mapx,mapy,tel,addr1,readcount,modifiedtime
394,영등포 여의도봄꽃축제,20210405,20210430,20.0,126.914133,37.532032,02-2670-3114,서울특별시 영등포구 여의서로,746805.0,20210401172258
537,창덕궁 달빛기행,20210429,20210620,23.0,126.991339,37.582825,02-3210-3503,서울특별시 종로구 율곡로,700585.0,20210504164714
704,[명예대표축제]진주 남강유등축제 2020,20201001,20201011,13.0,128.080276,35.189746,055-755-9111,경상남도 진주시 남강로 626,685539.0,20201201165059
405,오색별빛정원전 2021,20201204,20210314,1.0,127.352585,37.74484,1544-6703,경기도 가평군 상면 수목원로 432,575986.0,20201130161907
576,태안 세계튤립꽃박람회,20210409,20210510,14.0,126.33678,36.501102,041-675-5533,충청남도 태안군 안면읍 꽃지해안로 400,564948.0,20210413091916
...,...,...,...,...,...,...,...,...,...,...
452,이순신장군배 국제요트대회 2020,20201104,20201108,17.0,128.433863,34.82777,055)643-3335,경상남도 통영시 도남로 269-20,0.0,20201028181410
464,익산식품대전,20200904,20200910,9.0,127.082922,35.9795610080,NaN,온라인개최,0.0,20210621171209
74,국제해양영화제,20210722,20210725,16.0,129.127152,35.17112,051-790-9413,부산광역시 해운대구 수영강변대로 120,NaN,20210715111602
135,대한민국 청년미술축제,20210716,20210730,18.0,127.096301,37.502278,070-7938-5480,서울특별시 송파구 백제고분로 270,NaN,20210715173502


In [15]:
#결측값 확인
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 734 entries, 394 to 691
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           734 non-null    object 
 1   eventstartdate  734 non-null    int64  
 2   eventenddate    734 non-null    int64  
 3   sigungucode     727 non-null    float64
 4   mapx            702 non-null    object 
 5   mapy            702 non-null    object 
 6   tel             725 non-null    object 
 7   addr1           733 non-null    object 
 8   readcount       731 non-null    float64
 9   modifiedtime    734 non-null    int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 63.1+ KB


In [16]:
data.isnull().sum()

title              0
eventstartdate     0
eventenddate       0
sigungucode        7
mapx              32
mapy              32
tel                9
addr1              1
readcount          3
modifiedtime       0
dtype: int64

In [27]:
# 결측값 인덱스 색출 - reset_index 후 색출
# data = data.reset_index()
rows_with_nan = [index for index, row in data[['sigungucode']].iterrows() if row.isnull().any()]
rows_with_nan

[29, 124, 617, 648, 295, 77, 49]

In [35]:
# fill nan
# sigungucode / map x / map y 는 코드를 짜서 api에 집어넣어 추출, 그다음 병합하는 방법을 찾아봐야함.
# 현재로선 잘 못하겠음.. 하..

data['readcount'] = data['readcount'].fillna(0).round(0)
data['readcount'] = data['readcount'].astype('int')
data.isnull().sum()
data['sigungucode'] =data['sigungucode'].fillna(method='ffill')
data['mapx']=data['mapx'].fillna(method='ffill')
data['mapy']=data['mapy'].fillna(method='ffill')

In [36]:
print(data.iloc[526])
#'index 526 in online'
# fill in addr1 = online
# fill mapx mapy = 0,0
data['addr1'] = data['addr1'].fillna('online')
data['mapx'][526] = 0
data['mapy'][526] = 0
data['sigungucode'][526] = 0

title                파주 눈내리는 초리골 축제 2021
eventstartdate                  20210104
eventenddate                    20210214
sigungucode                         27.0
mapx                      126.8906847990
mapy                           37.852242
tel                         031-958-8880
addr1             경기도 파주시 법원읍 초리골길 138-9
readcount                              0
modifiedtime              20210119105734
Name: 585, dtype: object


<ipython-input-36-7048a3f88df4>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['mapx'][526] = 0
<ipython-input-36-7048a3f88df4>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['mapy'][526] = 0
<ipython-input-36-7048a3f88df4>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sigungucode'][526] = 0


In [40]:
data = data.set_index('title')


KeyError: "None of ['title'] are in the columns"

In [42]:
data = data.sort_values('readcount',ascending=False)

In [43]:
data

,eventstartdate,eventenddate,sigungucode,mapx,mapy,tel,addr1,readcount,modifiedtime
title,,,,,,,,,
영등포 여의도봄꽃축제,20210405,20210430,20.0,126.914133,37.532032,02-2670-3114,서울특별시 영등포구 여의서로,746805,20210401172258
창덕궁 달빛기행,20210429,20210620,23.0,126.991339,37.582825,02-3210-3503,서울특별시 종로구 율곡로,700585,20210504164714
[명예대표축제]진주 남강유등축제 2020,20201001,20201011,13.0,128.080276,35.189746,055-755-9111,경상남도 진주시 남강로 626,685539,20201201165059
오색별빛정원전 2021,20201204,20210314,1.0,127.352585,37.74484,1544-6703,경기도 가평군 상면 수목원로 432,575986,20201130161907
태안 세계튤립꽃박람회,20210409,20210510,14.0,126.33678,36.501102,041-675-5533,충청남도 태안군 안면읍 꽃지해안로 400,564948,20210413091916
...,...,...,...,...,...,...,...,...,...
섬 겨울꽃 랜선 축제 2020,20201211,20210131,12.0,126.251971,34.843629,061-240-8778,전라남도 신안군 압해읍 수락길 330,0,20210517162446
급이 있는 영화제 2020,20201126,20201127,16.0,129.127113,35.171075,051-990-7200,온라인개최,0,20210517162949
기획오페라 <사랑의 묘약> 2021,20210128,20210130,5.0,128.5922570230,35.883507,053-666-6170,대구광역시 북구 호암로 15,0,20210128165716


In [44]:
#차후 프로젝트에 해당 장소 + 시간때를 확인하여 행사가 있던 정보를 알려줌.